基于Module类的模型构造方法

In [1]:
# 创建模型参数,定义前向计算
# 无须定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的backward函数

import torch
from torch import nn

class MLP(nn.Module):
    def __init__(self, **kwargs):   # **kwargs:关键词参数
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256)
        self.act = nn.ReLU()
        self.out = nn.Linear(256, 10)

    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.out(a)


In [2]:
# 实例化MLP,进行前向计算
X = torch.rand(2, 784)
print(X)
net = MLP()
print(net)
net(X)  # 输入/输出的行数相同


tensor([[0.1790, 0.1814, 0.2331,  ..., 0.5106, 0.6714, 0.7915],
        [0.1670, 0.8081, 0.8338,  ..., 0.5298, 0.4352, 0.3658]])
MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (out): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-1.3406e-01, -2.8529e-01,  1.5584e-01,  7.3435e-03, -1.7915e-03,
         -3.9888e-02, -1.1606e-01, -2.2512e-01, -3.3228e-03, -6.4690e-02],
        [-7.5832e-02, -1.7804e-01,  1.8581e-01, -1.2697e-01, -1.1709e-01,
          4.4901e-03, -5.3200e-02, -2.0616e-01,  9.7260e-05, -1.8985e-01]],
       grad_fn=<AddmmBackward>)

使用sequential类构造模型
前面已经讲过

使用ModuleList类构造模型
可以像list一样进行append和extend操作
**但是**ModuleList仅仅是一个储存各种模块的列表，这些模块之间没有联系也没有顺序（所以不用保证相邻层的输入输出维度匹配），而且没有实现forward功能需要自己实现

In [3]:
net = nn.ModuleList([nn.Linear(784,256), nn.ReLU()])
net.append(nn.Linear(256, 10))
print(net[-1])  # 通过索引进行访问
print(net)







Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


使用get_constant函数,创建训练中的超参常数参数(不变,不可训练)

In [4]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__()

        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)
        # torch.mm:矩阵乘法操作

        x = self.linear(x)  #复用全连接层,两个全连接层共享参数

        # 控制流
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10

        return x.sum()

In [5]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(2.7593, grad_fn=<SumBackward0>)